# Build Image

In this notebook, we show the following steps for deploying a web service using AML:

- Create an image
- Test image locally


In [1]:
import docker
import matplotlib.pyplot as plt
import numpy as np
import requests
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azureml.core.model import Model
from azureml._model_management._util import (get_docker_client, pull_docker_image)
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.image import ContainerImage
from dotenv import get_key, find_dotenv
from testing_utilities import to_img, plot_predictions, get_auth, wait_until_ready


In [2]:
env_path = find_dotenv(raise_error_if_not_found=True)

In [3]:
resource_group = get_key(env_path, 'resource_group')
model_name = 'resnet_model'
image_name = get_key(env_path, 'image_name')

## Get workspace
Load existing workspace from the config file info.

In [4]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config(auth=get_auth())
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

workspace
azuremlresourcegroup
eastus
32cf04de-62b6-46b8-b31a-863d7fd95678


## Create Image

In [5]:
# create yml file to be used in the image
conda_pack = ["tensorflow-gpu==1.14.0"]
requirements = ["keras==2.2.0","Pillow==5.2.0", "azureml-defaults", "azureml-contrib-services" ,"toolz==0.9.0"] 

imgenv = CondaDependencies.create(conda_packages=conda_pack,pip_packages=requirements)
with open("img_env.yml", "w") as f:
    f.write(imgenv.serialize_to_string())

In [6]:

image_config = ContainerImage.image_configuration(execution_script = "driver.py",
                                                  runtime = "python",
                                                  conda_file = "img_env.yml",
                                                  description = "Image for AKS Deployment Tutorial",
                                                  tags = {"name":"AKS","project":"AML"}, 
                                                  dependencies = ["resnet152.py"],
                                                  enable_gpu = True
                                                 )


In [7]:
# create image. It may take upto 15-20 minutes. 
image = ContainerImage.create(name = image_name,
                              # this is the model object
                              models = [ws.models[model_name]],                              
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

Creating image
Running....................................................................................................................................................................
Succeeded
Image creation operation finished for image kerasimage:1, operation "Succeeded"


In [7]:
# You can find the logs of image creation
# image.image_build_log_uri

# You can get the image object when not creating a new image
image = ws.images['kerasimage']

## Test image locally
- Pull the image from ACR registry to local host 
- Start a container
- Test API call

In [8]:
ws = Workspace.from_config(auth=get_auth())
# Getting your container details
container_reg = ws.get_details()["containerRegistry"]
reg_name=container_reg.split("/")[-1]
container_url = "\"" + image.image_location + "\","
subscription_id = ws.subscription_id

client = ContainerRegistryManagementClient(ws._auth,subscription_id)
result= client.registries.list_credentials(resource_group, reg_name, custom_headers=None, raw=False)
username = result.username
password = result.passwords[0].value
print('ContainerURL:{}'.format(image.image_location))
print('Servername: {}'.format(reg_name))
print('Username: {}'.format(username))
print('Password: {}'.format(password))

ContainerURL:workspace0deabba7.azurecr.io/kerasimage:1
Servername: workspace0deabba7
Username: workspace0deabba7
Password: Jw5mTEOc/Yku9PVdzq8/Lzx/uIsMSrP/


In [9]:
dc = get_docker_client()

In [10]:
pull_docker_image(dc, image.image_location, username, password)

Pulling image from ACR (this may take a few minutes depending on image size)...

1: Pulling from kerasimage
Digest: sha256:f69dba33ca1171f75b71d955a21f311593c25a2ae2fd76071c29c287ad9df073
Status: Image is up to date for workspace0deabba7.azurecr.io/kerasimage:1


In [11]:
image.image_location

'workspace0deabba7.azurecr.io/kerasimage:1'

In [12]:
# make sure port 80 is not occupied
container_labels = {'containerName': 'kerasgpu'}
container = dc.containers.run(image.image_location, 
                                         detach=True, 
                                         ports={'5001/tcp': 80},
                                         labels=container_labels,
                                         runtime='nvidia' )

In [22]:
!which nvidia-container-runtime

/usr/bin/nvidia-container-runtime


In [23]:
!apt install nvidia-container-runtime

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [13]:
for log_msg in container.logs(stream=True):
    str_msg = log_msg.decode('UTF8')
    print(str_msg)
    if "Model loading time:" in str_msg:
        print('Model loaded and container ready')
        break

2020-04-20T11:50:41,893393522+00:00 - nginx/run 

2020-04-20T11:50:41,892172354+00:00 - gunicorn/run 

2020-04-20T11:50:41,893621716+00:00 - rsyslog/run 

2020-04-20T11:50:41,900579533+00:00 - iot-server/run 

EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...

2020-04-20T11:50:41,968059658+00:00 - iot-server/finish 1 0

2020-04-20T11:50:41,969181628+00:00 - Exit code 1 is normal. Not restarting iot-server.

Starting gunicorn 19.9.0

Listening at: http://127.0.0.1:31311 (10)

Using worker: sync

worker timeout is set to 300

Booting worker with pid: 49

Initializing logger

Starting up app insights client

Starting up request id generator

Starting up app insight hooks

Invoking user's init function

2020-04-20 11:50:48,495 | azureml.core.run | DEBUG | Could not load run context RunEnvironmentException:

	Message: Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run.

	InnerException N

In [27]:
client = docker.APIClient()
details = client.inspect_container(container.id)

In [30]:
dc.APIClient()

AttributeError: 'DockerClient' object has no attribute 'APIClient'

In [28]:
details['NetworkSettings']['Ports']

{}

In [17]:
service_ip = details['NetworkSettings']['Ports']['5001/tcp'][0]['HostIp']
service_port = details['NetworkSettings']['Ports']['5001/tcp'][0]['HostPort']

KeyError: '5001/tcp'

Wait a few seconds for the application to spin up and then check that everything works.

In [ ]:
print('Checking service on {} port {}'.format(service_ip, service_port))

In [ ]:
endpoint="http://__service_ip:__service_port"
endpoint = endpoint.replace('__service_ip', service_ip)
endpoint = endpoint.replace('__service_port', service_port)

max_attempts = 50
output_str = wait_until_ready(endpoint, max_attempts)
print(output_str)

In [ ]:
!curl 'http://{service_ip}:{service_port}/'

In [ ]:
IMAGEURL = "https://bostondata.blob.core.windows.net/aksdeploymenttutorialaml/220px-Lynx_lynx_poing.jpg"

In [ ]:
plt.imshow(to_img(IMAGEURL))

In [ ]:
with open('220px-Lynx_lynx_poing.jpg', 'rb') as f:
    img_data = f.read()

In [ ]:
%time r = requests.post('http://0.0.0.0:80/score', files={'image': img_data})
print(r)
r.json()

In [ ]:
images = (
    "https://bostondata.blob.core.windows.net/aksdeploymenttutorialaml/220px-Lynx_lynx_poing.jpg",
    "https://bostondata.blob.core.windows.net/aksdeploymenttutorialaml/Roadster_2.5_windmills_trimmed.jpg",
    "https://bostondata.blob.core.windows.net/aksdeploymenttutorialaml/Harmony_of_the_Seas_(ship,_2016)_001.jpg",
)

In [ ]:
from testing_utilities import read_image_from

In [ ]:
url = "http://0.0.0.0:80/score"
results = [
    requests.post(url, files={'image': read_image_from(img).read()}) for img in images
]

In [ ]:
plot_predictions(images, results)

In [ ]:
image_data = list(map(lambda img: read_image_from(img).read(), images)) # Retrieve the images and data

In [ ]:
timer_results = list()
for img in image_data:
    res=%timeit -r 1 -o -q requests.post(url, files={'image': img})
    timer_results.append(res.best)

In [ ]:
timer_results

In [ ]:
print("Average time taken: {0:4.2f} ms".format(10 ** 3 * np.mean(timer_results)))

In [31]:
container.stop()

In [32]:
# remove stopped container
!docker system prune -f

Deleted Containers:
d3cc0103bfb5e40b011c807534ca1f8fd5e4a82d7da13e139c3e17be95f48791
e26ed9a82b2fa819954f5106df9ac85fc343b2edae0932d056f24cdad3bee3fe
66e8e0d4af177415df9b552f023d83205c4a8614650bb01d73ae19ded72db00f
44464e170e5403f43cb8cfb69437b54a176cc5a63314a286b54dd7f3ea9c0dee
1bf383737ee2754347a110b592094ae820dd3a177deab0bf056cf9009c2512ef

Total reclaimed space: 71.72MB


We can now move on to [Create kubenetes cluster and deploy web service](04_DeployOnAKS.ipynb) with the image we just built.